In [ ]:
import  numpy as np
import pandas as pd
import tensorflow as tf

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn import pipeline

from tqdm import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU: ', tpu.master())
    
except Exception as e:
    tpu = None
    

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    
else:
    strategy = tf.distribute.get_strategy()
    
print('Number of clusters: ', strategy.num_replicas_in_sync)

### Data Loading

In [ ]:
train = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv')

In [ ]:
train.head()

In [ ]:
train.drop(['severe_toxic','obscene','threat','insult','identity_hate'],axis=1,inplace=True)
train.shape

In [ ]:
train = train.loc[:209999, :]

### Data Cleaning

In [ ]:
contractions_dict = {     
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I had",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "iit will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that had",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there had",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they had",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [ ]:
import re
def clean(text, contractions=contractions_dict, remove_stop=False):
    text = text.lower()
    text = re.sub(r'\([^)]*\)', '', text)
    text = ' '.join([contractions[t] if t in contractions else t for t in text.split(' ')])
    text = re.sub(r"'s\b", "", text)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = re.sub('[m]{2, }', 'mm', text)
    
    return ' '.join(text.strip().split())

In [ ]:
print('* BEFORE CLEANING: ', train.comment_text.iloc[0], '\n')
print('* AFTER CLEANING: ', clean(train.comment_text.iloc[0]))

In [ ]:
train['comment_text_clean'] = train['comment_text'].progress_apply(clean)

In [ ]:
max_len = int(train['comment_text_clean'].str.split().apply(len).describe()['75%'] + 26)
print('Max comment length: ', max_len)

### Data Preparation

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(list(train['comment_text_clean']))

In [ ]:
X_train_seq = tokenizer.texts_to_sequences(train['comment_text'])
X_train_pad = tf.keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=max_len)

y_train = train.toxic.values
X_train_pad.shape, y_train.shape

### Setting up a callback

In [ ]:
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)]

### A basic RNN

In [ ]:
%time
with strategy.scope():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=(max_len, )))
    model.add(tf.keras.layers.Embedding(len(tokenizer.word_index)+1, 300, input_length=max_len))
    model.add(tf.keras.layers.SimpleRNN(512))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
model.summary()

In [ ]:
model.fit(X_train_pad, y_train, 
          epochs=5, 
          validation_split=0.2, 
          batch_size=50*strategy.num_replicas_in_sync,
          callbacks=callbacks)

In [ ]:
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.plot(model.history.history['accuracy'])
plt.title('Simple RNN - Training Accuracy')

plt.subplot(2, 2, 2)
plt.plot(model.history.history['val_accuracy'])
plt.title('Simple RNN - Testing Accuracy')

plt.subplot(2, 2, 3)
plt.plot(model.history.history['loss'], c='r')
plt.title('Simple RNN - Training Loss')

plt.subplot(2, 2, 4)
plt.plot(model.history.history['val_loss'], c='r')
plt.title('Simple RNN - Testing Loss')

plt.show()

In [ ]:
glove_file = '/kaggle/input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl'
glove = np.load(glove_file, allow_pickle=True)

embedding_matrix = np.zeros((len(tokenizer.word_index)+1, 300))
for word, i in tqdm(tokenizer.word_index.items()):
    embedding_vector = glove.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

### Using stacked LSTMs with pre-trained word embeddings

In [ ]:
with strategy.scope():
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Embedding(len(tokenizer.word_index)+1,
                                        embedding_matrix.shape[1],
                                        weights=[embedding_matrix],
                                        input_length=max_len,
                                        trainable=False))
    
    model.add(tf.keras.layers.LSTM(512,
                                   return_sequences=True,
                                   recurrent_dropout=0.3))
    model.add(tf.keras.layers.LSTM(512, 
                                   recurrent_dropout=0.3))
    
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.002),
                  metrics=['accuracy'])
    
model.summary()

In [ ]:
model.fit(X_train_pad, y_train, 
          epochs=10, 
          validation_split=0.2, 
          batch_size=500*strategy.num_replicas_in_sync,
          callbacks=callbacks)

In [ ]:
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.plot(model.history.history['accuracy'])
plt.title('LSTM - Training Accuracy')

plt.subplot(2, 2, 2)
plt.plot(model.history.history['val_accuracy'])
plt.title('LSTM - Testing Accuracy')

plt.subplot(2, 2, 3)
plt.plot(model.history.history['loss'], c='r')
plt.title('LSTM - Training Loss')

plt.subplot(2, 2, 4)
plt.plot(model.history.history['val_loss'], c='r')
plt.title('LSTM - Testing Loss')

plt.show()

### Using stacked GRUs

In [ ]:
with strategy.scope():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Embedding(len(tokenizer.word_index)+1,
                                        embedding_matrix.shape[1],
                                        weights=[embedding_matrix],
                                        input_length=max_len,
                                        trainable=False))
    
    model.add(tf.keras.layers.GRU(512, recurrent_dropout=0.3, return_sequences=True))
    model.add(tf.keras.layers.GRU(512))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.002),
                  metrics=['accuracy'])
    
model.summary()

In [ ]:
model.fit(X_train_pad, y_train, 
          epochs=10, 
          validation_split=0.2, 
          batch_size=500*strategy.num_replicas_in_sync,
          callbacks=callbacks)

In [ ]:
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.plot(model.history.history['accuracy'])
plt.title('GRU - Training Accuracy')

plt.subplot(2, 2, 2)
plt.plot(model.history.history['val_accuracy'])
plt.title('GRU - Testing Accuracy')

plt.subplot(2, 2, 3)
plt.plot(model.history.history['loss'], c='r')
plt.title('GRU - Training Loss')

plt.subplot(2, 2, 4)
plt.plot(model.history.history['val_loss'], c='r')
plt.title('GRU - Testing Loss')

plt.show()

### Using a Bi-LSTM

In [ ]:
with strategy.scope():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Embedding(len(tokenizer.word_index)+1,
                                        embedding_matrix.shape[1],
                                        weights=[embedding_matrix],
                                        input_length=max_len,
                                        trainable=False))
    
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, recurrent_dropout=0.3)))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.002),
                  metrics=['accuracy'])
    
model.summary()

In [ ]:
model.fit(X_train_pad, y_train, 
          epochs=5, 
          validation_split=0.2, 
          batch_size=60*strategy.num_replicas_in_sync,
          callbacks=callbacks)

In [ ]:
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.plot(model.history.history['accuracy'])
plt.title('Bi-LSTM - Training Accuracy')

plt.subplot(2, 2, 2)
plt.plot(model.history.history['val_accuracy'])
plt.title('Bi-LSTM - Testing Accuracy')

plt.subplot(2, 2, 3)
plt.plot(model.history.history['loss'], c='r')
plt.title('Bi-LSTM - Training Loss')

plt.subplot(2, 2, 4)
plt.plot(model.history.history['val_loss'], c='r')
plt.title('Bi-LSTM - Testing Loss')

plt.show()

### Using a Seq2Seq Model with Attention

In [ ]:
import transformers

In [ ]:
test_data = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
test_data['comment_text_clean'] = test_data['comment_text'].apply(clean)
test_data.head()

In [ ]:
def fast_encode(texts, tokenizer, chunk_size=512, maxlen=max_len):
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(max_length=maxlen)
    all_ids = []
    
    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encodings = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encodings])
        
    return np.array(all_ids)

In [ ]:
AUTO_TUNE = tf.data.experimental.AUTOTUNE
EPOCHS = 10
BATCH_SIZE = 120 * strategy.num_replicas_in_sync
MAX_LEN = max_len

In [ ]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
tokenizer.save_pretrained('.')

fast_tokenizer = transformers.BertTokenizerFast('vocab.txt', lowercase=False)
fast_tokenizer

In [ ]:
X_train = fast_encode(train.comment_text_clean.astype(str), fast_tokenizer.tokenizer, maxlen=MAX_LEN)
y_train = train.toxic.values

X_test = fast_encode(test_data.comment_text_clean.astype(str), fast_tokenizer.tokenizer, maxlen=MAX_LEN)
y_test = test_data.toxic.values

X_train[1]

In [ ]:
train_dataset = (tf.data.Dataset
                .from_tensor_slices((X_train, y_train))
                .repeat()
                .shuffle(2048)
                .batch(BATCH_SIZE)
                .prefetch(AUTO_TUNE))

test_dataset = (tf.data.Dataset
                .from_tensor_slices((X_test, y_test))
                .repeat()
                .shuffle(2048)
                .cache()
                .batch(BATCH_SIZE)
                .prefetch(AUTO_TUNE))

In [ ]:
def build_model(transformer, max_len=MAX_LEN):
    input_word_ids = tf.keras.layers.Input(shape=(max_len, ), dtype=tf.int32, name='input_word_ids')
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = tf.keras.layers.Dense(1, activation='sigmoid')(cls_token)
    
    model = tf.keras.models.Model(inputs=input_word_ids, outputs=out)
    
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
with strategy.scope():
    transformer_layer = (
        transformers.TFDistilBertModel
        .from_pretrained('distilbert-base-multilingual-cased')
    )
    
    model = build_model(transformer_layer, max_len=MAX_LEN)
    
model.summary()

In [ ]:
n_steps = X_train.shape[0] // BATCH_SIZE
history = model.fit(train_dataset,
                    steps_per_epoch=n_steps,
                    epochs=EPOCHS)

In [ ]:
n_steps = X_test.shape[0] // BATCH_SIZE
history_test = model.fit(test_dataset,
                    steps_per_epoch=n_steps,
                    epochs=EPOCHS)

In [ ]:
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.plot(history.history['accuracy'])
plt.title('DistilBERT - Training Accuracy')

plt.subplot(2, 2, 2)
plt.plot(history.history['loss'])
plt.title('DistilBERT - Training Loss')

plt.subplot(2, 2, 3)
plt.plot(history_test.history['accuracy'], c='r')
plt.title('DistilBERT - Testing Accuracy')

plt.subplot(2, 2, 4)
plt.plot(history_test.history['loss'], c='r')
plt.title('DistilBERT - Testing Loss')

plt.show()